# Flower 연합학습 전략 구현을 통한 FedAvg 넘어서기

Flower를 활용한 KAICD 연합학습 강좌 2부에 오신 것을 환영합니다!

이번 튜토리얼에서는 지난 시간에 구축한 연합학습 시스템을 커스터마이징 할 것입니다(이번에도 FLower와 PyTorch를 사용합니다!).

Part 1에서는 PyTorch를 활용하여 모델 훈련 파이프라인과 데이터 불러오기를 구성합니다.

Part 2에서는 Flower를 활용하여 Part 1에서 구현한 PyTorch 기반 파이프라인을 가지고 연합학습을 진행합니다.

## Part 0: 사전준비

시작하기에 앞서, Google Colab GPU 가속 설정을 확인해야 합니다.

`런타임 > 런타임 유형 변경 > 하드웨어 가속: GPU > 저장`

### 의존성 설치

다음으로 필요한 패키지를 설치하고 가져옵니다.

In [1]:
!pip install torch==1.9.0 torchvision==0.10.0 git+https://github.com/adap/flower.git@release/0.17#egg=flwr["simulation"]

from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

  Cloning https://github.com/adap/flower.git (to revision release/0.17) to /tmp/pip-install-8xe726tq/flwr_fc849276817e453a932d10376268a4f7
  Running command git clone -q https://github.com/adap/flower.git /tmp/pip-install-8xe726tq/flwr_fc849276817e453a932d10376268a4f7
  Running command git checkout -b release/0.17 --track origin/release/0.17
  Switched to a new branch 'release/0.17'
  Branch 'release/0.17' set up to track remote branch 'release/0.17' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 49.6 MB 16 kB/s 
     |████████████████████████████████| 72 kB 727 kB/s 
     |████████████████████████████████| 128 kB 40.0 MB/s 
     |████████████████████████████████| 78 kB 9.0 MB/s 
     |████████████████████████████████| 3.2 MB 43.1 MB/s 
     |████████████████████████████████| 1.3 MB 61.8 MB/s 
     |████████████████████████████████| 65 kB 5.1 MB/s 
     

Google Colab에서 실행하고 런타임에 GPU 가속기가 있다면, 출력 결과에서 `Training on cuda:0` 문장을 확인할 수 있습니다.

### 데이터 불러오기

이제 CIFAR-10 훈련 및 테스트 데이터 세트를 불러오고, 10개의 작은 데이터 세트로 분할하여(각각 훈련 및 검증 데이터 세트로 분할) `DataLoader`로 포장합니다:

In [2]:
NUM_CLIENTS = 10

def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


### 모델 훈련/평가

기본적인 모델을 정의하고, 훈련 및 테스트 함수를 작성합니다:

In [3]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(testloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## Part 1: 연합학습 전략 커스터마이징

### Flower 클라이언트

Flower 클라이언트를 구현하기 위해 `flwr.client.NumPyClient`의 하위 클래스를 생성하고,

세 가지 함수(`get_parameters`, `fit`, `evaluate`)를 구현합니다:

In [4]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

### 커스터마이징 전략으로 연합학습 시작

우리는 이제 클라이언트 측의 훈련과 평가를 정의하는 `FlowerClient`와 

Flower가 특정 클라이언트에게 `fit` 또는 `evaluate`가 필요할 때마다 `FlowerClient`를 만들 수 있는 `client_fn`를 갖게 됐습니다!

마지막 단계는 `flwr.simulation.start_simulation`을 이용하여 실제 시뮬레이션을 시작하는 겁니다.

`start_simulation` 함수는 `FlowerClient` 인스턴스를 생성하는 데 사용되는 `client_fn`,

연합학습 시뮬레이션에 참여하는 클라이언트의 수 `num_clients`,

연합학습 학습 횟수 `num_rounds`, 연합학습 전략 등 여러가지 기능을 포함하고 있습니다.

여기서 전략이라고 칭하는 것은 *Federated Averaging*(FedAvg)와 같은 연합학습 접근법 및 알고리즘을 캡슐화합니다.

Flower에는 다양한 연합학습 전략들이 내장되어 있지만, 자신만의 전략을 구현하고 사용할 수 있습니다.

이번 튜토리얼에서는 FedAvg를 기본으로 사용하며, 몇가지 기본 파라미터들을 조정하여 커스터마이징합니다.

마지막 단계는 시뮬레이션을 시작하는 `start_simulation`을 실제로 호출하는 겁니다:

In [5]:
# # Create FedAvg strategy
# strategy=fl.server.strategy.FedAdam(
#         fraction_fit=1.0,  # Sample 10% of available clients for training
#         fraction_eval=1.0,  # Sample 5% of available clients for evaluation
#         min_fit_clients=10,  # Never sample less than 10 clients for training
#         min_eval_clients=10,  # Never sample less than 5 clients for evaluation
#         min_available_clients=10,  # Wait until all 10 clients are available
# )

# # Start simulation
# fl.simulation.start_simulation(
#     client_fn=client_fn,
#     num_clients=NUM_CLIENTS,
#     num_rounds=5,
#     strategy=strategy,
# )

## Part 2: 서버 파라미터 초기화

Flower는 기본적으로 하나의 임의 클라이언트에 초기 파라미터를 요청하여 전역 모델을 초기화합니다.

그러나, 대부분의 경우 파라미터 초기화에 대한 더 많은 제어를 필요로 합니다.

따라서 Flower는 초기 파라미터를 전략에 직접 전달할 수 있는 기능을 구현해놨습니다:

In [ ]:
# Pass initial parameters to the Strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.3,
    min_fit_clients=3,
    min_eval_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.weights_to_parameters(
        get_parameters(Net())
    ),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,  # Just three rounds
    strategy=strategy,
)

INFO flower 2021-10-25 01:41:35,348 | app.py:95 | Ray initialized with resources: {'node:172.28.0.2': 1.0, 'object_store_memory': 3921749606.0, 'CPU': 2.0, 'accelerator_type:P100': 1.0, 'memory': 7843499214.0, 'GPU': 1.0}
INFO flower 2021-10-25 01:41:35,354 | app.py:104 | Starting Flower simulation running: {'num_rounds': 3}
INFO flower 2021-10-25 01:41:35,355 | server.py:118 | Initializing global parameters
INFO flower 2021-10-25 01:41:35,359 | server.py:300 | Using initial parameters provided by strategy
INFO flower 2021-10-25 01:41:35,361 | server.py:120 | Evaluating initial parameters
INFO flower 2021-10-25 01:41:35,363 | server.py:133 | FL starting
DEBUG flower 2021-10-25 01:41:35,365 | server.py:255 | fit_round: strategy sampled 3 clients (out of 10)


`FedAvg` 전략에 `initial_parameters`를 전달하면 Flower는 클라이언트에게 초기 파라미터를 요청하지 않습니다.

로그를 자세히 살펴보면, `FlowerClient.get_parameters` 함수에 대한 통신이 전혀 이루어지지 않았음을 알 수 있습니다.

## Part 3: 서버 파라미터 **평가** ("중앙집중형 평가")

Flower는 서버 또는 클라이언트 측의 집계된 모델을 평가할 수 있습니다.

서버와 클라이언트 측 평가는 어떤 면에선 비슷하다고 할 수 있지만, 다른 면도 존재합니다.

**중앙집중형 평가**는 간단한 개념입니다.

중앙집중형 기계학습에서 평가하는 것과 같은 방식으로 작동합니다.

평가 목적으로 사용할 수 있는 서버 데이터 세트가 있다면, 제일 좋겠죠?

위의 경우에는 클라이언트에게 모델을 보내지 않고도 훈련 각 라운드 후에 새로 집계된 모델을 평가할 수 있습니다.

또한, 전체 평가 데이터 세트를 항상 사용할 수 있다는 장점이 있습니다.

**연합 평가**는 보다 복잡하지만, 더 강력한 측면이 있습니다:

1. 중앙집중형 데이터 세트가 필요하지 않습니다.
2. 더 큰 데이터 세트에 걸쳐 모델을 평가할 수 있게 해줍니다.
3. 더 현실적인 평가 결과를 산출할 수 있습니다.

사실, 우리가 대표적인 평가 결과를 얻기를 원한다면 많은 시나리오들이 **연합 평가**를 사용하길 요구합니다.

하지만, 이런 강력한 기능을 사용하는 데에는 많은 비용이 듭니다.

우선 클라이언트 측에서 평가하기 시작하면, 우리는 평가 데이터 세트가 연속적인 학습에 따라 종종 변화한다는 것을 인지해야 합니다.

모델을 변경하지 않더라도, 클라이언트가 항상 연결되어 있지 않고 각 클라이언트의 데이터 세트가 변경될 수 있기 때문에

시간이 지남에 따라 평가 결과가 변동(불안정한 평가 결과)하는 것을 관찰할 수 있습니다.

우리는 `FlowerClient`에서 평가 함수를 실행함으로써, 클라이언트 측에서 연합 평가가 어떻게 작용하는지 살펴봤습니다.

이제 서버 측에서 평가할 수 있는 방법을 살펴보겠습니다:

In [ ]:
# Create a function that returns an evaluation function
def get_evaluation_fn():
    # Load data and model here to avoid the overhead of doing it in `evaluate` itself
    net = Net()
    valloader = valloaders[0]

    # The `evaluate` function will be called after every round
    def evaluate(
        weights: fl.common.Weights,
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        set_parameters(net, weights)  # Update model with the latest parameters
        loss, accuracy = test(net, valloader)
        print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
        return loss, {"accuracy": accuracy}

    return evaluate

In [ ]:
# Pass the evaluation function to the Strategy
strategy=fl.server.strategy.FedAdam(
    fraction_fit=1.0,
    fraction_eval=1.0,
    min_fit_clients=10,
    min_eval_clients=10,
    min_available_clients=10,
    initial_parameters=fl.common.weights_to_parameters(
        get_parameters(Net())
    ),
    eval_fn=get_evaluation_fn(),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=3,
    strategy=strategy,
)

## Part 4: 서버에서 클라이언트로 값 전송
TODO

## Part 4: 사용자 정의 전략 구현

TODO

## 요약

이번 튜토리얼에서는 연합학습 전략을 커스터마이징하고, 다른 전략도 선택해보며 서버 파라미터를 초기화하고,

서버 측의 모델도 평가해봄으로써 시스템을 점진적으로 향상시킬 수 있는 방법을 살펴봤습니다.

**정말 적은 양의 코드에 꽤 많은 힘이 있습니다!**